In [ ]:
# Ejercicio 1

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str) + "\n")

query1 = db.proveedores.find(
    {"activo": True, "habilitado": True},
    {"_id": 0, "CUIT": 1,"razon_social": 1, "tipo_sociedad": 1, "direccion": 1, "activo": 1, "habilitado": 1,"telefonos": 1}
)


save_to_txt("query1.txt", list(query1))


In [ ]:
# Ejercicio 1

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str) + "\n")

query1 = db.proveedores.find(
    {"activo": True, "habilitado": True},
    {"_id": 0, "CUIT": 1,"razon_social": 1, "tipo_sociedad": 1, "direccion": 1, "activo": 1, "habilitado": 1,"telefonos": 1}
)


save_to_txt("query1.txt", list(query1))


In [ ]:
#   Ejercicio 2

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

db = client["mi_base"]
proveedores = db["proveedores"]

query2 = proveedores.find(
    {"razon_social": {"$regex": "Tecnología", "$options": "i"}},
    {"_id": 0, "CUIT": 1, "telefonos": 1}
)

save_to_txt("query2.txt", list(query2))
      

In [ ]:
#   Ejercicio 3

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

db = client["mi_base"]
proveedores = db["proveedores"]

query3 = proveedores.aggregate([
    {"$unwind": "$telefonos"},
    {"$project": {
        "_id": 0,
        "razon_social": 1,
        "CUIT": 1,
        "telefono": "$telefonos"
    }}
])
save_to_txt("query3.txt", list(query3))

In [ ]:
# Ejercicio 4

from neo4j import GraphDatabase

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def get_proveedores_with_orders(tx):
    query = """
    MATCH (p:Proveedor)<-[:ES_DE]-(pedido:Pedido)
    RETURN DISTINCT p;
    """
    result = tx.run(query)
    return [record["p"] for record in result]

with driver.session() as session:
    query4 = session.execute_read(get_proveedores_with_orders)

save_to_txt("query4.txt", query4)


In [ ]:
#   Ejercicio 5

from neo4j import GraphDatabase

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_proveedores_sin_ordenes(tx):
    result = tx.run("""
        MATCH (p:Proveedor) 
        WHERE NOT ((:Pedido)-[:ES_DE]->(p))
        RETURN p.id_proveedor AS id_proveedor,
               p.CUIT AS CUIT_proveedor,
               p.razon_social AS razon_social,
               p.tipo_sociedad AS tipo_sociedad,
               p.direccion AS direccion,
               p.activo AS activo,
               p.habilitado AS habilitado
    """)
    return [
        {
            "id_proveedor": record["id_proveedor"],
            "CUIT_proveedor": record["CUIT_proveedor"],
            "razon_social": record["razon_social"],
            "tipo_sociedad": record["tipo_sociedad"],
            "direccion": record["direccion"],
            "activo": record["activo"],
            "habilitado": record["habilitado"]
        } 
        for record in result
    ]


with driver.session() as session:
    query5 = session.execute_read(obtener_proveedores_sin_ordenes)

save_to_txt("query5.txt", query5)

print(f"✅ Se guardaron {len(query5)} proveedores sin órdenes en 'query5.txt'.")

In [ ]:
#   Ejercicio 6

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")

db = client["mi_base"]
proveedores = db["proveedores"]

query6 = proveedores.aggregate([
    {"$lookup": {
        "from": "pedidos",
        "localField": "_id",
        "foreignField": "id_proveedor",
        "as": "ordenes"
    }},
    {"$project": {
        "_id": 0,
        "id_proveedor": "$_id",
        "razon_social": 1,
        "total_ordenes": {"$size": "$ordenes"},
        "monto_total_sin_iva": {
            "$sum": {"$map": {
                "input": "$ordenes",
                "as": "orden",
                "in": "$$orden.total_sin_iva"
            }}
        },
        "monto_total_con_iva": {
            "$sum": {
                "$map": {
                    "input": "$ordenes",
                    "as": "orden",
                    "in": {
                        "$add": ["$$orden.total_sin_iva", "$$orden.iva"]
                    }
                }
            }
        }
    }}
])
save_to_txt("query6.txt", list(query6))

In [ ]:
# Ejercicio 7

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

query7 = db.pedidos.aggregate([
    {
        "$lookup": {
        "from": "proveedores",
        "localField": "id_proveedor",
        "foreignField": "_id",
        "as": "proveedor"
        }
    },
    { "$unwind": "$proveedor" },
    {
        "$match": {
            "proveedor.CUIT": 30660608175
        }
    },
    {
        "$project": {
            "_id": 0,
            "fecha": 1,
            "total_sin_iva": 1,
            "iva": 1,
            "detalle": 1
        }
    }
])

save_to_txt("query7.txt", query7)


In [ ]:
#   Ejercicio 8

from neo4j import GraphDatabase


uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_productos_pedidos_al_menos_una_vez(tx):
    result = tx.run("""
        MATCH (:Pedido)-[:INCLUYE]->(p:Producto)
        RETURN DISTINCT p.id_producto AS id_producto,
               p.descripcion AS descripcion,
               p.marca AS marca,
               p.categoria AS categoria,
               p.precio AS precio,
               p.stock_actual AS stock_actual,
               p.stock_futuro AS stock_futuro
    """)
    return [
        {
            "id_producto": record["id_producto"],
            "descripcion": record["descripcion"],
            "marca": record["marca"],
            "categoria": record["categoria"],
            "precio": record["precio"],
            "stock_actual": record["stock_actual"],
            "stock_futuro": record["stock_futuro"]
        }
        for record in result
    ]

with driver.session() as session:
    query8 = session.execute_read(obtener_productos_pedidos_al_menos_una_vez)

save_to_txt("query8.txt", query8)

In [ ]:
#   Ejercicio 9

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
pedidos = db["pedidos"]

query9 = pedidos.aggregate([
    {"$unwind": "$detalle"},
    {"$lookup": {
        "from": "productos",
        "localField": "detalle.id_producto",
        "foreignField": "_id",
        "as": "producto"
    }},
    {"$unwind": "$producto"},
    {"$match": {
        "producto.marca": "COTO"
    }},
    {"$project": {
        "_id": 1,
        "fecha": 1,
        "id_proveedor": 1,
        "total_sin_iva": 1,
        "iva": 1,
        "detalle": 1,
        "producto": 1
    }}
])
save_to_txt("query9.txt", list(query9))


In [ ]:
# Ejercicio 10 

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

db.create_collection("ordenes_pedido_vista", viewOn="pedidos", pipeline=[
    {
        "$lookup": {
            "from": "proveedores",
            "localField": "id_proveedor",
            "foreignField": "_id",
            "as": "proveedor"
        }
    },
    { "$unwind": "$proveedor" },
    {
        "$project": {
            "_id": 0,
            "razon_social": "$proveedor.razon_social",
            "fecha": 1,
            "iva": 1,
            "total_sin_iva": 1,
            "total_con_iva": {
                "$add": ["$total_sin_iva", "$iva"]
            },
            "detalle": 1
        }
    },
    {
        "$sort": {
            "fecha": 1
        }
    }
])

# Consultar la vista
query10 = db.ordenes_pedido_vista.find()

# Escribir el resultado en el archivo (sobrescribiendo si ya existe)
save_to_txt("query10.txt", query10)




In [ ]:
#   Ejercicio 11

from neo4j import GraphDatabase

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_productos_no_pedidos(tx):
    result = tx.run("""
        MATCH (p:Producto)
        WHERE NOT EXISTS((:Pedido)-[:INCLUYE]->(p))
        RETURN p.id_producto AS id_producto,
               p.descripcion AS descripcion,
               p.marca AS marca,
               p.categoria AS categoria,
               p.precio AS precio,
               p.stock_actual AS stock_actual,
               p.stock_futuro AS stock_futuro
    """)
    return [
        {
            "id_producto": record["id_producto"],
            "descripcion": record["descripcion"],
            "marca": record["marca"],
            "categoria": record["categoria"],
            "precio": record["precio"],
            "stock_actual": record["stock_actual"],
            "stock_futuro": record["stock_futuro"]
        } for record in result
    ]


with driver.session() as session:
    productos_no_pedidos = session.execute_read(obtener_productos_no_pedidos)

save_to_txt("query11.txt", productos_no_pedidos)

In [ ]:
#   Ejercicio 12

from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

db.command({
    "create": "proveedores_inhabilitados_activos",
    "viewOn": "proveedores",
    "pipeline": [
        {"$match": {
            "activo": True,
            "habilitado": False
        }},
        {"$project": {
            "_id": 0,
            "id_proveedor": "$_id",
            "razon_social": 1,
            "CUIT": 1,
            "direccion": 1,
            "tipo_sociedad": 1
        }}
    ]
})


## info de la vista en query12.txt
vista = db["proveedores_inhabilitados_activos"]
rta = list(vista.find())
save_to_txt("query12.txt", rta)


In [ ]:
# Ejercicio 13 - Crear un proveedor

from pymongo import MongoClient, errors as mongo_errors
from neo4j import GraphDatabase, exceptions as neo4j_errors

# Conexión a Mongo
client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

# Conexión a Neo4j
uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

# Función para validar la lista de teléfonos
# Retorna True si es válida, False y un mensaje de error si no lo es.
def validar_telefonos(telefonos):
    if not isinstance(telefonos, list):
        return False, "La lista de teléfonos no es válida."

    numeros_vistos = set()

    for t in telefonos:
        if not isinstance(t, dict):
            return False, f"Teléfono inválido (no es un diccionario): {t}"

        if "codigo_area" not in t or not isinstance(t["codigo_area"], int):
            return False, f"Código de área inválido: {t}"

        if "nro_telefono" not in t or not isinstance(t["nro_telefono"], str):
            return False, f"Número de teléfono inválido: {t}"

        if "tipo" not in t or not isinstance(t["tipo"], str):
            return False, f"Tipo de teléfono inválido: {t}"

        clave = (t["codigo_area"], t["nro_telefono"])
        if clave in numeros_vistos:
            return False, f"Teléfono duplicado: {clave}"
        numeros_vistos.add(clave)

    return True, "OK"  

# Función genérica para crear proveedor
def create_proveedor(proveedor):
    es_valido, mensaje = validar_telefonos(proveedor["telefonos"])
    if not es_valido:
        raise ValueError(f"Error en formato de teléfonos: {mensaje}")
    
    proveedor["_id"] = proveedor["id_proveedor"]
    try:
        db.proveedores.insert_one(proveedor)
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al insertar en MongoDB: {e}")
    
    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    CREATE (p:Proveedor {
                        id_proveedor: $id_proveedor,
                        CUIT: $CUIT,
                        razon_social: $razon_social,
                        tipo_sociedad: $tipo_sociedad,
                        direccion: $direccion,
                        activo: $activo,
                        habilitado: $habilitado
                    })
                """,
                id_proveedor=proveedor["id_proveedor"],
                CUIT=proveedor["CUIT"],
                razon_social=proveedor["razon_social"],
                tipo_sociedad=proveedor["tipo_sociedad"],
                direccion=proveedor["direccion"],
                activo=proveedor["activo"],
                habilitado=proveedor["habilitado"]
                )

                for tel in proveedor["telefonos"]:
                    tx.run("""
                        MERGE (t:Telefono {
                            codigo_area: $codigo_area,
                            nro_telefono: $nro_telefono
                        })
                        SET t.tipo = $tipo
                        WITH t
                        MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                        MERGE (p)-[:TIENE_TELEFONO]->(t)
                    """, codigo_area=tel["codigo_area"],
                        nro_telefono=tel["nro_telefono"],
                        tipo=tel["tipo"],
                        id_proveedor=proveedor["id_proveedor"])
            
            session.execute_write(neo4j_tx)        
        print(f"Proveedor {proveedor['_id']} registrado correctamente en ambas bases.")
    except neo4j_errors.Neo4jError as e:
        db.proveedores.delete_one({"_id": proveedor["id_proveedor"]})
        raise RuntimeError(f"Error al insertar en Neo4j, se revirtió la inserción en MongoDB: {e}")  

proveedor_nuevo = {
    "id_proveedor": 9999,
    "CUIT": "30-99999999-9",
    "razon_social": "Proveedor Test",
    "tipo_sociedad": "SA",
    "direccion": "Av. Siempre Viva 742",
    "activo": True,
    "habilitado": True,
    "telefonos": [
        {"codigo_area": 11, "nro_telefono": "12345678", "tipo": "fijo"},
        {"codigo_area": 11, "nro_telefono": "87654321", "tipo": "celular"}
    ]
}
create_proveedor(proveedor_nuevo)
                
      


In [ ]:
# Ejercicio 13 - Modificar un proveedor

from pymongo import MongoClient, errors as mongo_errors
from neo4j import GraphDatabase, exceptions as neo4j_errors

# Conexión a Mongo
client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

# Conexión a Neo4j
uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

# Función para modificar proveedor
def update_proveedor(proveedor):
    id_proveedor = proveedor.get("id_proveedor")
    if id_proveedor is None:
        raise ValueError("Debe proporcionar 'id_proveedor' en los datos para identificar el proveedor.")
    
    es_valido, mensaje = validar_telefonos(proveedor["telefonos"])
    if not es_valido:
        raise ValueError(f"Error en formato de teléfonos: {mensaje}")

    proveedor_actual = db.proveedores.find_one({"_id": proveedor["id_proveedor"]})
    if not proveedor:
        raise ValueError("Proveedor no encontrado.")    

    proveedor["_id"] = proveedor["id_proveedor"]
    try:
        db.proveedores.update_one({"_id": id_proveedor}, {"$set": proveedor})
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al actualizar en MongoDB: {e}")    

    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                    SET p.CUIT = COALESCE($CUIT, p.CUIT),
                        p.razon_social = COALESCE($razon_social, p.razon_social),
                        p.tipo_sociedad = COALESCE($tipo_sociedad, p.tipo_sociedad),
                        p.direccion = COALESCE($direccion, p.direccion),
                        p.activo = COALESCE($activo, p.activo),
                        p.habilitado = COALESCE($habilitado, p.habilitado)
                """, id_proveedor=id_proveedor,
                     CUIT=proveedor.get("CUIT"),
                     razon_social=proveedor.get("razon_social"),
                     tipo_sociedad=proveedor.get("tipo_sociedad"),
                     direccion=proveedor.get("direccion"),
                     activo=proveedor.get("activo"),
                     habilitado=proveedor.get("habilitado"))

                if proveedor.get("telefonos", None) is not None:
                    tx.run("""
                        MATCH (p:Proveedor {id_proveedor: $id_proveedor})-[r:TIENE_TELEFONO]->(t:Telefono)
                        DELETE r
                    """, id_proveedor=id_proveedor)

                    for tel in proveedor.get("telefonos"):
                        tx.run("""
                            MERGE (t:Telefono {
                                codigo_area: $codigo_area,
                                nro_telefono: $nro_telefono
                            })
                            SET t.tipo = $tipo
                            WITH t
                            MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                            MERGE (p)-[:TIENE_TELEFONO]->(t)
                        """, codigo_area=tel["codigo_area"],
                             nro_telefono=tel["nro_telefono"],
                             tipo=tel["tipo"],
                             id_proveedor=id_proveedor)

            session.execute_write(neo4j_tx)
        print(f"Proveedor {id_proveedor} modificado correctamente en ambas bases.")
    except neo4j_errors.Neo4jError as e:
        try:
            db.proveedores.replace_one({"_id": id_proveedor}, proveedor_actual)
        except mongo_errors.PyMongoError as re:
            raise RuntimeError(f"Error al revertir MongoDB tras fallo en Neo4j: {re}") from e
        print(f"Error al actualizar proveedor: {e}")
        raise RuntimeError(f"Error al actualizar en Neo4j, se revirtió la actualización en MongoDB: {e}")

# Ejemplo de datos para actualizar:
datos_actualizacion = {
    "id_proveedor": 9999,                      
    "CUIT": "30-12345678-9",         
    "razon_social": "Proveedor Actualizado S.A.",
    "direccion": "Calle Falsa 123",
    "activo": True,
    "habilitado": True,
    "telefonos": [
        {"tipo": "fijo", "codigo_area": 11, "nro_telefono": "44445555"},
        {"tipo": "celular", "codigo_area": 15, "nro_telefono": "1555555555"}
    ]
}
update_proveedor(datos_actualizacion) 

In [ ]:
# Ejercicio 13 - Eliminar un proveedor

from pymongo import MongoClient, errors as mongo_errors
from neo4j import GraphDatabase, exceptions as neo4j_errors

# Conexión a Mongo
client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

# Conexión a Neo4j
uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

# Función para eliminar proveedor
def delete_proveedor(id_proveedor):
    proveedor_actual = db.proveedores.find_one({"_id": id_proveedor})
    if not proveedor_actual:
        raise ValueError("Proveedor no encontrado.")
    
    try:
        db.proveedores.delete_one({"_id": id_proveedor})
    except mongo_errors.PyMongoError as e:
        raise RuntimeError(f"Error al eliminar en MongoDB: {e}")
    
    try:
        with driver.session() as session:
            def neo4j_tx(tx):
                tx.run("""
                    MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                    DETACH DELETE p
                """, id_proveedor=id_proveedor)

            session.execute_write(neo4j_tx)

        print(f"Proveedor {id_proveedor} eliminado correctamente en ambas bases.")
    except neo4j_errors.Neo4jError as e:
        try:
            db.proveedores.insert_one(proveedor_actual)
        except mongo_errors.PyMongoError as re:
            raise RuntimeError(f"Error al revertir MongoDB tras fallo en Neo4j: {re}") from e
        print(f"Error al eliminar proveedor: {e}")
        raise RuntimeError(f"Error al eliminar en Neo4j, se revirtió la eliminación en MongoDB: {e}")

# Ejemplo de datos para eliminar:
id_proveedor = 9999
delete_proveedor(id_proveedor) 


In [ ]:
#   Ejercicio 14


In [19]:
# Ejercicio 14 - Crear nuevo producto

import json
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from neo4j import GraphDatabase

# Conexiones
mongo_client = MongoClient("mongodb://mongo:27017")
db = mongo_client["mi_base"]
col_productos = db["productos"]

neo4j_uri = "bolt://neo4j:7687"
neo4j_user = "neo4j"
neo4j_pass = "longerpass123"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))

def crear_producto(producto_data):
    """Crea un producto en Mongo y Neo4j de manera transaccional, manejando duplicados."""
    mongo_result = None
    try:
        try:
            mongo_result = col_productos.insert_one({
                "_id": producto_data["id_producto"],
                "descripcion": producto_data["descripcion"],
                "marca": producto_data["marca"],
                "categoria": producto_data["categoria"],
                "precio": producto_data["precio"],
                "stock_actual": producto_data["stock_actual"],
                "stock_futuro": producto_data["stock_futuro"]
            })
        except DuplicateKeyError:
            print(f"El producto con id {producto_data['id_producto']} ya existe en MongoDB.")
            return

        with driver.session() as session:
            session.run("""
                CREATE (p:Producto {
                    id_producto: $id_producto,
                    descripcion: $descripcion,
                    marca: $marca,
                    categoria: $categoria,
                    precio: $precio,
                    stock_actual: $stock_actual,
                    stock_futuro: $stock_futuro
                })
            """, **producto_data)

        print(f"Producto {producto_data['id_producto']} creado correctamente en ambas bases.")
    except Exception as e:
        # Si falla en Neo4j, hacemos rollback en Mongo
        if mongo_result:
            col_productos.delete_one({"_id": producto_data["id_producto"]})
        print(f"Error al crear producto: {e}")

# Ejemplo de uso
producto_nuevo = {
    "id_producto": 1001,
    "descripcion": "Producto de Prueba",
    "marca": "Acme",
    "categoria": "test",
    "precio": 123.45,
    "stock_actual": 100,
    "stock_futuro": 50
}
crear_producto(producto_nuevo)

✅ Producto 1001 creado correctamente en ambas bases.


In [23]:
# Ejercicio 14 - Modificar un producto existente

from pymongo import MongoClient
from neo4j import GraphDatabase

# Conexiones
mongo_client = MongoClient("mongodb://mongo:27017")
db = mongo_client["mi_base"]
col_productos = db["productos"]

neo4j_uri = "bolt://neo4j:7687"
neo4j_user = "neo4j"
neo4j_pass = "longerpass123"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))


def actualizar_producto(producto_data):
    """Actualiza un producto en Mongo y Neo4j de manera transaccional, con manejo de errores."""
    producto_id = producto_data["id_producto"]

    producto_actual_mongo = col_productos.find_one({"_id": producto_id})
    if not producto_actual_mongo:
        print(f"El producto con id {producto_id} NO existe en MongoDB.")
        return

    with driver.session() as session:
        result = session.run("""
            MATCH (p:Producto {id_producto: $id_producto})
            RETURN p
        """, id_producto=producto_id).single()
        if not result:
            print(f"El producto con id {producto_id} NO existe en Neo4j.")
            return

    try:
        col_productos.update_one({"_id": producto_id}, {"$set": {
            "descripcion": producto_data["descripcion"],
            "marca": producto_data["marca"],
            "categoria": producto_data["categoria"],
            "precio": producto_data["precio"],
            "stock_actual": producto_data["stock_actual"],
            "stock_futuro": producto_data["stock_futuro"]
        }})

        with driver.session() as session:
            session.run("""
                MATCH (p:Producto {id_producto: $id_producto})
                SET p.descripcion = $descripcion,
                    p.marca = $marca,
                    p.categoria = $categoria,
                    p.precio = $precio,
                    p.stock_actual = $stock_actual,
                    p.stock_futuro = $stock_futuro
            """, **producto_data)

        print(f"Producto {producto_id} actualizado correctamente en ambas bases.")
    except Exception as e:
        # Si falla en Neo4j, hacemos rollback en Mongo
        col_productos.update_one({"_id": producto_id}, {"$set": producto_actual_mongo})
        print(f"Error al actualizar producto: {e}")

# Ejemplo de uso
producto_editado = {
    "id_producto": 1009,
    "descripcion": "Producto de Prueba Editado",
    "marca": "Acme Edit",
    "categoria": "test",
    "precio": 150.0,
    "stock_actual": 80,
    "stock_futuro": 60
}
actualizar_producto(producto_editado)

✅ Producto 1009 actualizado correctamente en ambas bases.


In [1]:
#   Ejercicio 15

from pymongo import MongoClient
from neo4j import GraphDatabase
from datetime import datetime

mongo_client = MongoClient("mongodb://mongo:27017")
db = mongo_client["mi_base"]
col_proveedores = db["proveedores"]
col_pedidos = db["pedidos"]
col_productos = db["productos"]

neo4j_uri = "bolt://neo4j:7687"
neo4j_user = "neo4j"
neo4j_pass = "longerpass123"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))


"""
    Registra una nueva orden en MongoDB y Neo4j
    solo si el proveedor está activo y habilitado en MongoDB.
"""
def registrar_orden(orden_data):
    proveedor_id = orden_data["id_proveedor"]
    proveedor = col_proveedores.find_one({"_id": proveedor_id, "activo": True, "habilitado": True})

    if not proveedor:
        print(f"El proveedor {proveedor_id} no está activo y habilitado. No se registra la orden.")
        return

    ## Validaciones realizada en mongoDB - asumo consistente ambas bases
    total_sin_iva = 0.0
    productos_afectados = []  # Guardamos productos válidos y cantidades para actualizar stock después
    
    for item in orden_data["detalle"]:
        prod = col_productos.find_one({"_id": item["id_producto"]})
        if not prod:
            print(f"El producto {item['id_producto']} no existe en las bases de datos.")
            return
        if round(prod["precio"], 2) != round(item["precio"], 2):
            print(f"El precio del producto {item['id_producto']} no coincide. Bases de datos: {prod['precio']} - Orden: {item['precio']}")
            return
        if prod["stock_actual"] < item["cantidad"]:
            print(f"No hay suficiente stock para el producto {item['id_producto']}. Stock: {prod['stock_actual']} - Requiere: {item['cantidad']}")
            return
        total_sin_iva += item["precio"] * item["cantidad"]
        productos_afectados.append((item["id_producto"], item["cantidad"]))

    orden_data["total_sin_iva"] = round(total_sin_iva, 2)
    orden_data["iva"] = round(total_sin_iva * 0.21, 2)

    for item in orden_data["detalle"]:
        if not col_productos.find_one({"_id": item["id_producto"]}):
            print(f"El producto {item['id_producto']} no existe en la Base de Datos.")
            return

    # Registramos en MongoDB
    try:
        col_pedidos.insert_one(orden_data)
    except Exception as e:
        print(f"Error al insertar orden en MongoDB: {e}")
        return

    
    # Actualizamos stock en MongoDB
    try:
        for prod_id, cant in productos_afectados:
            producto = col_productos.find_one({"_id": prod_id})
            nuevo_stock = producto["stock_actual"] - cant
            col_productos.update_one({"_id": prod_id}, {"$set": {"stock_futuro": nuevo_stock}})
    except Exception as e:
        col_pedidos.delete_one({"_id": orden_data["_id"]})
        print(f"Error al actualizar stock futuro en MongoDB. Rollback orden. Detalle: {e}")
        return

    # Registramos en Neo4j
    try:
        with driver.session() as session:
            session.run("""
                MATCH (prov:Proveedor {id_proveedor: $prov_id})
                CREATE (p:Pedido {
                    id_pedido: $id,
                    fecha: $fecha,
                    total_sin_iva: $total_sin_iva,
                    iva: $iva
                })
                CREATE (prov)-[:HACE]->(p)
            """, prov_id=proveedor_id, id=orden_data["_id"],
                 fecha=orden_data["fecha"].strftime("%Y-%m-%d"),
                 total_sin_iva=orden_data["total_sin_iva"],
                 iva=orden_data["iva"])

            for item in orden_data["detalle"]:
                session.run("""
                    MATCH (p:Pedido {id_pedido: $id_pedido})
                    MATCH (prod:Producto {id_producto: $id_prod})
                    CREATE (p)-[:INCLUYE {
                        nro_item: $item,
                        cantidad: $cantidad
                    }]->(prod)
                """, id_pedido=orden_data["_id"],
                     id_prod=item["id_producto"],
                     item=item["nro_item"],
                     cantidad=item["cantidad"])

            # Actualizamos stock en Neo4j 
            for prod_id, cant in productos_afectados:
                session.run("""
                    MATCH (prod:Producto {id_producto: $id_prod})
                    SET prod.stock_futuro = prod.stock_actual - $cant
                """, id_prod=prod_id, cant=cant)

        print(f"Orden {orden_data['_id']} registrada correctamente en ambas bases.")
    except Exception as e:
        # Rollback en Mongo si falla Neo4j
        col_pedidos.delete_one({"_id": orden_data["_id"]})
        print(f"Error al registrar orden en Neo4j: {e}")



nueva_orden = {
    "_id": 400,
    "id_proveedor": 2,
    "fecha": datetime.strptime("2025-06-19", "%Y-%m-%d"),
    "detalle": [
        {"id_producto": 1, "nro_item": 1, "cantidad": 5, "precio": 443.88}
    ]
}

registrar_orden(nueva_orden)

Orden 400 registrada correctamente en ambas bases.
